# Limit of sum = sum of limits


This example mirrors the Lean proof from [Lean For the Curious Mathematician 2023](https://github.com/lftcm2023/lftcm2023/blob/master/LftCM/C02_Logic/S06_Sequences_and_Convergence.lean#L29).

```lean
theorem convergesTo_add {s t : ℕ → ℝ} {a b : ℝ}
      (cs : ConvergesTo s a) (ct : ConvergesTo t b) :
    ConvergesTo (fun n ↦ s n + t n) (a + b) := by
  intro ε εpos
  dsimp
  have ε2pos : 0 < ε / 2 := by linarith
  rcases cs (ε / 2) ε2pos with ⟨Ns, hs⟩
  rcases ct (ε / 2) ε2pos with ⟨Nt, ht⟩
  use max Ns Nt
  intro n hn
  have ngeNs : n ≥ Ns := le_of_max_le_left hn
  have ngeNt : n ≥ Nt := le_of_max_le_right hn
  calc
    |s n + t n - (a + b)| = |s n - a + (t n - b)| := by
      congr
      ring
    _ ≤ |s n - a| + |t n - b| := (abs_add _ _)
    _ < ε / 2 + ε / 2 := (add_lt_add (hs n ngeNs) (ht n ngeNt))
    _ = ε := by norm_num
```

In [1]:
from pylogic import *
settings['PYTHON_OPS_RETURN_PROPS'] = True

def ConvergesTo(s, a):
    return Limit.make_epsilon_N_definition(a, s)

In [ ]:
s, t = variables('s', 't', sequence=True, real=True)
a, b = variables('a', 'b', real=True)
s_plus_t = Sequence('(s+t)', nth_term = lambda n: s[n] + t[n])
cs = ConvergesTo(s, a).assume(); ct = ConvergesTo(t, b).assume()
to_prove = ConvergesTo(s_plus_t, a + b)
with AssumptionsContext() as ctx:
    ε = ctx_var(r'\epsilon', real=True, positive=True); εpos = (ε > 0).by_inspection()
    ε2 = ε / 2
    ε2pos = (ε2 > 0).by_inspection()

    Ns, (_, hs) = cs(ε2)(ε2pos).rename_variable('N_s')
    Nt, (_, ht) = ct(ε2)(ε2pos).rename_variable('N_t')
    N = Max(Ns, Nt)

    with AssumptionsContext() as n_ctx:
        n = ctx_var('n', natural=True)
        hn = (n >= N).assume()
        ngeNs = hn.transitive(N.geq_a)
        ngeNt = hn.transitive(N.geq_b)
        ineq0 = hs(n)(ngeNs); ineq1 = ht(n)(ngeNt)

        sn = s[n]
        tn = t[n]
        stn = s_plus_t[n]
        sn_tn = sn + tn
        eq2 = stn.equals(sn_tn).by_eval()
        
        ineq = transitive("<",
            (eq2 - (a + b)).apply(abs),
            abs(sn_tn - (a + b)).equals(abs((sn - a) + (tn - b))).by_eval(),
            Reals.theorems.triangle_inequality(sn - a, tn - b),
            ineq0 + ineq1,
            ((ε2 + ε2).equals(ε)).by_eval()
        )
        conclude(ineq)
    conclude(n_ctx.get_first_proven().thus_there_exists("N", N, Naturals))
convergesTo_add = ctx.get_first_proven()


True
